In [48]:
%pwd


'c:\\Users\\70544\\OneDrive\\Desktop\\projects\\AI-vs.-Human-generated-image-classification'

In [49]:
import os
os.chdir("c:\\Users\\70544\\OneDrive\\Desktop\\projects\\AI-vs.-Human-generated-image-classification")

In [50]:
%pwd

'c:\\Users\\70544\\OneDrive\\Desktop\\projects\\AI-vs.-Human-generated-image-classification'

In [51]:

from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    
from src.CNNclassifier.constants import *
from src.CNNclassifier.utils.common import read_yaml,create_directories
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.params=read_yaml(params_filepath)

        self.config=read_yaml(config_filepath)
        
        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
        root_dir=config.root_dir,
        source_url=config.source_url,
        local_data_file=config.local_data_file,
        unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [52]:
!pip install tensorflow pandas numpy dvc notebook numpy matplotlib seaborn python-box pyYAML tqdm ensure joblib types-pyYAML scipy Flask Flask-Cors requests


In [53]:
import os
import urllib.request as req
import requests
import zipfile
import io
import zipfile
from src.CNNclassifier import logger
from src.CNNclassifier.utils.common import get_size

In [54]:
import re

def extract_filename(content_disposition):
    match = re.search(r'filename\*?=(?:UTF-8\'\')?"?([^\";\n]+)"?', content_disposition)
    return match.group(1) if match else None
def get_confirm_token(response):
    for k, v in response.cookies.items():
        if k.startswith('download_warning'):
            return v
    return None
def download_from_gdrive_url(url, output_path):
    response = requests.get(url, stream=True)

    # Try to extract filename from headers if output_path not provided
    if output_path is None:
        content_disp = response.headers.get("Content-Disposition", "")
        output_path = extract_filename(content_disp) or "downloaded_file"

    # Save file
    with open(output_path, "wb") as f:
        for chunk in response.iter_content(32768):
            if chunk:
                f.write(chunk)

    print(f"Downloaded to: {output_path}")
    return output_path, response.headers
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=download_from_gdrive_url(self.config.source_url,self.config.local_data_file)
            logger.info(f"{filename} downloaded! with following info:\n {headers}")
            print(filename)
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [57]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-02 11:21:41,899: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-02 11:21:41,900: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-02 11:21:41,901: INFO: common: created directory at: artifacts]
[2025-07-02 11:21:41,902: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-02 11:21:41,902: INFO: 780166876: file already exists of size: ~ 4469551 KB]
